Part 1.2

Below is the functions we implemented to use in Part 2.2

In [ ]:
import random

# For demonstration, let's define some global parameters here.
# In practice, you might structure them differently or store them in a config class.
LAMBDA = 1.0               # arrival rate (patients/hour)
MU_T = 0.476190476         # triage nurse service rate
MU_S = 0.16                # stable home-care rate
MU_CB = 0.118518519        # hospital bed service rate
P_STABLE = 0.2             # Probability that a patient is stable
P_CRITICAL = 1 - P_STABLE  # Probability that a patient is critical

S = 3  # number of triage nurses
K = 9  # number of hospital beds

# For random number seeding, use your group's seed:
MY_SEED = 4040800189  # or sum of your group members' IDs
random.seed(MY_SEED)

def GenerateInterarrival():
    """
    Returns an exponentially distributed interarrival time
    with rate LAMBDA.
    """
    return random.expovariate(LAMBDA)

def GenerateNurseServiceTime():
    """
    Returns an exponentially distributed service time
    for triage nurses with rate MU_T.
    """
    return random.expovariate(MU_T)

def GenerateHospitalHealingTime():
    """
    Returns an exponentially distributed healing time
    for critical patients in a hospital bed (rate = MU_CB).
    """
    return random.expovariate(MU_CB)

def GenerateHomeHealingTime(condition='s'):
    """
    Returns an exponentially distributed healing time for a patient who is home.
      - If 's': stable patient at home, rate = MU_S.
      - If 'c': critical patient forced to go home. We first sample alpha ~ Uniform[1.25, 1.75].
        Then the rate = MU_CB / alpha, making the mean 1/(MU_CB/alpha) = alpha / MU_CB.
    """
    if condition == 's':
        # Stable patient at home
        return random.expovariate(MU_S)
    else:
        # Critical patient forced to go home
        alpha = random.uniform(1.25, 1.75)
        # The rate is scaled by alpha
        return random.expovariate(MU_CB / alpha)

    # Example event structure: (eventTime, eventType, patientID, extraInfo)
#   eventTime : the simulation time at which event occurs
#   eventType : e.g. 'Arrival', 'DepartureTriage', 'TreatedAtHospital', 'RecoveryHome'
#   patientID : an identifier for the patient (optional, but useful)
#   extraInfo : any other data you need (e.g. stable/critical, etc.)

# We'll keep a global list (or priority queue) for the FEL:
FEL = []

# State variables:
currentTime = 0.0
numberInTriageQueue = 0
busyNurses = 0
occupiedBeds = 0

# Additional counters/statistics, such as:
totalPatientsArrived = 0
totalPatientsHealed = 0
rejectedCriticalCount = 0

# And so forth...

def Arrival(event):
    """
    Handle the arrival of a new patient.
    event is assumed to be (time, 'Arrival', patientID, {})
    """
    global currentTime, numberInTriageQueue, busyNurses, totalPatientsArrived

    # 1) Update simulation clock to this event's time
    currentTime = event[0]

    # 2) Increment counters
    totalPatientsArrived += 1

    # 3) Schedule the next arrival
    interarrival_time = GenerateInterarrival()
    next_arrival_time = currentTime + interarrival_time
    FEL.append((next_arrival_time, 'Arrival', None, {}))

    # 4) Check if a triage nurse is free
    if busyNurses < S:
        # Nurse is available immediately
        busyNurses += 1
        service_time = GenerateNurseServiceTime()
        departure_triage_time = currentTime + service_time

        # We'll store patient info in extraInfo if needed
        FEL.append((departure_triage_time, 'DepartureTriage', None, {}))
    else:
        # No free nurse, patient must join the queue
        numberInTriageQueue += 1

def DepartureTriage(event):
    """
    Handle a patient's completion of triage service.
    event is assumed to be (time, 'DepartureTriage', patientID, extraInfo)
    """
    global currentTime, numberInTriageQueue, busyNurses, occupiedBeds, rejectedCriticalCount

    # 1) Update clock
    currentTime = event[0]

    # 2) If there is a queue waiting, immediately take next patient for triage
    if numberInTriageQueue > 0:
        numberInTriageQueue -= 1
        # Start service for next patient
        service_time = GenerateNurseServiceTime()
        departure_time = currentTime + service_time
        FEL.append((departure_time, 'DepartureTriage', None, {}))
    else:
        # Nurse becomes idle
        busyNurses -= 1

    # 3) Determine if this departing patient is stable or critical
    if random.random() < P_STABLE:
        # Stable -> schedule home recovery event
        home_time = GenerateHomeHealingTime(condition='s')
        FEL.append((currentTime + home_time, 'RecoveryHome', None, {'isStable': True}))
    else:
        # Critical
        if occupiedBeds < K:
            # Admit to hospital bed
            occupiedBeds += 1
            hospital_healing_time = GenerateHospitalHealingTime()
            FEL.append((currentTime + hospital_healing_time, 'TreatedAtHospital', None, {}))
        else:
            # Forced home (bed not available)
            rejectedCriticalCount += 1
            home_time = GenerateHomeHealingTime(condition='c')
            FEL.append((currentTime + home_time, 'RecoveryHome', None, {'isStable': False}))

def TreatedAtHospital(event):
    """
    Handle a critical patient's discharge from the hospital.
    event is (time, 'TreatedAtHospital', patientID, extraInfo)
    """
    global currentTime, occupiedBeds

    # 1) Update clock
    currentTime = event[0]

    # 2) One bed becomes free
    occupiedBeds -= 1

    # 3) Patient leaves system - if you want to record stats about total heal times,
    #    you'd do it here (e.g., event[0] - patient.arrival_time, etc.)
    #    but for now, just note that the patient is done.

    # No further events for this patient.
    pass
def RecoveryHome(event):
    """
    Handle a patient's completion of home-care.
    event is (time, 'RecoveryHome', patientID, extraInfo)
    """
    global currentTime

    # 1) Update clock
    currentTime = event[0]

    # 2) This patient leaves the system. We can record stats if needed, e.g. total time in system.
    #    If extraInfo['isStable'] is True, we know it's a stable patient, else a forced-home critical.

    pass
def get_next_event():
    """
    Pop the soonest event from the FEL.
    """
    # Make sure your FEL is sorted by time
    FEL.sort(key=lambda e: e[0])
    return FEL.pop(0)  # remove and return the first event

def main_simulation_loop(max_events=50):
    """
    A simple loop that processes events until max_events are executed,
    or you can stop by another condition (e.g., number of recovered patients).
    """
    event_count = 0
    while event_count < max_events and len(FEL) > 0:
        next_event = get_next_event()
        event_type = next_event[1]

        if event_type == 'Arrival':
            Arrival(next_event)
        elif event_type == 'DepartureTriage':
            DepartureTriage(next_event)
        elif event_type == 'TreatedAtHospital':
            TreatedAtHospital(next_event)
        elif event_type == 'RecoveryHome':
            RecoveryHome(next_event)
        else:
            # Unrecognized event
            pass

        event_count += 1

Part 2.2

Initial conditions can be adjusted in the code according to the preferences.

In [ ]:
import random
import math

# ===============================
# GLOBAL PARAMETERS
# ===============================
LAMBDA = 1.0          # Arrival rate
MU_T = 0.476190476    # Triage nurse service rate
MU_S = 0.16           # Stable home-care rate
MU_CB = 0.118518519   # Hospital bed service rate
P_STABLE = 0.2        # Probability stable
P_CRITICAL = 0.8      # Probability critical

S = 3   # Number of triage nurses
K = 9   # Number of hospital beds

SEED = 4040800189


# ===============================
# RANDOM VARIATE GENERATORS
# ===============================
def GenerateInterarrival():
    return random.expovariate(LAMBDA)

def GenerateNurseServiceTime():
    return random.expovariate(MU_T)

def GenerateHospitalHealingTime():
    return random.expovariate(MU_CB)

def GenerateHomeHealingTime(condition='s'):
    """
    condition='s' => stable (Exp(MU_S))
    condition='c' => forced-home critical => Exp(MU_CB/alpha)
    """
    if condition == 's':
        return random.expovariate(MU_S)
    else:
        alpha = random.uniform(1.25, 1.75)
        return random.expovariate(MU_CB / alpha)


# ===============================
# GLOBAL STATE & STRUCTURES
# ===============================
state = {}
FEL = []
event_history = []

# Tracking each patient's arrival_time, etc.
patient_info = {}
next_patient_id = 0


# ===============================
# SCHEDULE / GET EVENTS
# ===============================
def schedule_event(time, etype, patientID=None, extra=None):
    FEL.append((time, etype, patientID, extra))

def get_next_event():
    FEL.sort(key=lambda x: x[0])
    return FEL.pop(0)

# ===============================
# EVENT FUNCTIONS
# ===============================
def Arrival(event):
    """A new patient arrives to triage (or queue)."""
    global state, FEL, next_patient_id, patient_info

    state['clock'] = event[0]

    pid = next_patient_id
    next_patient_id += 1
    # Record arrival time
    patient_info[pid] = {'arrival_time': state['clock']}

    state['totalArrivals'] += 1

    # Schedule next arrival
    next_arr_time = state['clock'] + GenerateInterarrival()
    schedule_event(next_arr_time, 'Arrival')

    # Check if nurse free
    if state['busyNurses'] < S:
        state['busyNurses'] += 1
        finish_time = state['clock'] + GenerateNurseServiceTime()
        schedule_event(finish_time, 'DepartureTriage', pid)
    else:
        state['numberInTriageQueue'] += 1


def DepartureTriage(event):
    """A patient finishes triage and is either stable or critical."""
    global state, FEL, patient_info

    current_time = event[0]
    state['clock'] = current_time
    pid = event[2]  # who finished triage

    # If triage queue > 0, immediately start next triage
    if state['numberInTriageQueue'] > 0:
        state['numberInTriageQueue'] -= 1
        # We'll create a new patient ID or re-use logic (in reality you store the queue).
        # For simplicity, let's create a new ID with partial data.
        global next_patient_id
        new_pid = next_patient_id

        next_patient_id += 1
        patient_info[new_pid] = {'arrival_time': current_time}  # minimal

        finish2 = current_time + GenerateNurseServiceTime()
        schedule_event(finish2, 'DepartureTriage', new_pid)
    else:
        # Freed a nurse
        state['busyNurses'] -= 1

    # Classify stable vs. critical
    if random.random() < P_STABLE:
        # stable
        state['stableCount'] += 1
        done_home = current_time + GenerateHomeHealingTime('s')
        schedule_event(done_home, 'RecoveryHome', pid, {'isStable': True})
    else:
        # critical
        state['criticalCount'] += 1
        if state['occupiedBeds'] < K:
            state['occupiedBeds'] += 1
            done_bed = current_time + GenerateHospitalHealingTime()
            schedule_event(done_bed, 'TreatedAtHospital', pid)
        else:
            # forced home
            state['rejectedCritical'] += 1
            done_home = current_time + GenerateHomeHealingTime('c')
            schedule_event(done_home, 'RecoveryHome', pid, {'isStable': False})


def TreatedAtHospital(event):
    """A critical patient finishes hospital care."""
    global state, FEL, patient_info
    current_time = event[0]
    state['clock'] = current_time
    pid = event[2]

    state['occupiedBeds'] -= 1
    state['patientsHealed'] += 1

    # record time in system
    arr_time = patient_info[pid]['arrival_time'] if pid in patient_info else 0.0
    state['sum_time_in_system'] += (current_time - arr_time)
    state['count_patients_finished'] += 1


def RecoveryHome(event):
    """A stable or forced-home critical finishes home care."""
    global state, FEL, patient_info
    current_time = event[0]
    state['clock'] = current_time
    pid = event[2]

    state['patientsHealed'] += 1

    # record time in system
    arr_time = patient_info[pid]['arrival_time'] if pid in patient_info else 0.0
    state['sum_time_in_system'] += (current_time - arr_time)
    state['count_patients_finished'] += 1

# ===============================
# APPLY INITIAL CONDITIONS
# (Schedules "already-busy" resources)
# ===============================
def apply_initial_condition(initial_condition):
    """
    If 'half': schedule half triage nurses & half beds as busy from time 0,
               each with a random finishing time.
    If 'full': schedule all triage nurses & all beds as busy from time 0.
    This ensures they eventually free up,
    instead of staying occupied forever.
    """
    global state, FEL, next_patient_id, patient_info

    if initial_condition == 'half':
        # half nurses
        half_nurses = math.floor(S/2)
        for i in range(half_nurses):
            pid = next_patient_id
            next_patient_id += 1
            # patient arrived at time 0
            patient_info[pid] = {'arrival_time': 0.0}

            finish_t = random.expovariate(MU_T)
            schedule_event(finish_t, 'DepartureTriage', pid)
        state['busyNurses'] = half_nurses

        # half beds
        half_beds = math.floor(K/2)
        for i in range(half_beds):
            pid = next_patient_id
            next_patient_id += 1
            patient_info[pid] = {'arrival_time': 0.0}  # started hospital at time 0

            finish_b = random.expovariate(MU_CB)
            schedule_event(finish_b, 'TreatedAtHospital', pid)
        state['occupiedBeds'] = half_beds

    elif initial_condition == 'full':
        # all nurses
        for i in range(S):
            pid = next_patient_id
            next_patient_id += 1
            patient_info[pid] = {'arrival_time': 0.0}

            finish_t = random.expovariate(MU_T)
            schedule_event(finish_t, 'DepartureTriage', pid)
        state['busyNurses'] = S

        # all beds
        for i in range(K):
            pid = next_patient_id
            next_patient_id += 1
            patient_info[pid] = {'arrival_time': 0.0}

            finish_b = random.expovariate(MU_CB)
            schedule_event(finish_b, 'TreatedAtHospital', pid)
        state['occupiedBeds'] = K
    else:
        # 'empty' do nothing
        pass


# ===============================
# MAIN SIMULATION
# ===============================
def run_simulation(target_healed, max_events, initial_condition):
    """
    Runs the simulation until 'target_healed' patients have completed.
    Returns (event_history, results_dict).
    """
    global state, FEL, event_history, patient_info, next_patient_id

    # Reset everything
    random.seed(SEED)  # or remove if you want different seeds per run
    state = {
        'clock': 0.0,
        'numberInTriageQueue': 0,
        'busyNurses': 0,
        'occupiedBeds': 0,
        'patientsHealed': 0,
        'totalArrivals': 0,
        'rejectedCritical': 0,
        'stableCount': 0,
        'criticalCount': 0,
        'sum_time_in_system': 0.0,
        'count_patients_finished': 0,
    }
    FEL = []
    event_history = []
    patient_info = {}
    next_patient_id = 0

    # Time-based accumulators
    last_event_time = 0.0
    total_time_triage_empty = 0.0
    total_time_beds_empty = 0.0
    total_time_both_empty = 0.0
    nurse_busy_time = 0.0
    beds_occupied_time = 0.0

    # 1) Schedule the first arrival
    first_arrival = GenerateInterarrival()
    schedule_event(first_arrival, 'Arrival')

    # 2) Apply initial condition:
    apply_initial_condition(initial_condition)

    # 3) Main loop
    num_events = 0
    while state['patientsHealed'] < target_healed and num_events < max_events and len(FEL) > 0:
        e = get_next_event()
        current_time = e[0]
        event_type = e[1]

        delta = current_time - last_event_time

        # Accumulate time-based stats
        # triage empty => busyNurses==0 and queue==0
        if state['busyNurses']==0 and state['numberInTriageQueue']==0:
            total_time_triage_empty += delta
        # beds empty => occupiedBeds==0
        if state['occupiedBeds']==0:
            total_time_beds_empty += delta
        # both empty
        if (state['busyNurses']==0 and state['numberInTriageQueue']==0
            and state['occupiedBeds']==0):
            total_time_both_empty += delta

        # usage
        nurse_busy_time += (state['busyNurses'] * delta)
        beds_occupied_time += (state['occupiedBeds'] * delta)

        # Update clock
        last_event_time = current_time

        # Process
        if event_type == 'Arrival':
            Arrival(e)
        elif event_type == 'DepartureTriage':
            DepartureTriage(e)
        elif event_type == 'TreatedAtHospital':
            TreatedAtHospital(e)
        elif event_type == 'RecoveryHome':
            RecoveryHome(e)
        else:
            pass

        num_events += 1

        # Record first 20 events
        if num_events <= 20:
            event_history.append({
                'Event#': num_events,
                'Clock': round(state['clock'], 4),
                'EventType': event_type,
                'TriageQueue': state['numberInTriageQueue'],
                'BusyNurses': state['busyNurses'],
                'OccupiedBeds': state['occupiedBeds'],
                'PatientsHealed': state['patientsHealed']
            })

    # End loop
    sim_time = state['clock']

    # Now compute final measures

    # 1) Prob triage empty
    prob_triage_empty = total_time_triage_empty/sim_time if sim_time>0 else 0.0
    # 2) Prob beds empty
    prob_beds_empty = total_time_beds_empty/sim_time if sim_time>0 else 0.0
    # 3) Prob both empty
    prob_both_empty = total_time_both_empty/sim_time if sim_time>0 else 0.0

    # 4) Average nurse util
    avg_nurse_util = (nurse_busy_time/(sim_time * S)) if sim_time>0 else 0.0
    # 5) Average #occupied beds
    avg_occupied_beds = (beds_occupied_time/sim_time) if sim_time>0 else 0.0

    # 6) Proportion critical rejected
    if state['criticalCount']>0:
        crit_reject_rate = state['rejectedCritical']/state['criticalCount']
    else:
        crit_reject_rate = 0.0

    # 7) Proportion treated at home
    forced_home = state['rejectedCritical']
    stable_home = state['stableCount']
    total_home = forced_home + stable_home
    if state['totalArrivals']>0:
        prop_home = total_home / state['totalArrivals']
    else:
        prop_home = 0.0

    # 8) Average time in system
    if state['count_patients_finished']>0:
        avg_time_sys = state['sum_time_in_system']/state['count_patients_finished']
    else:
        avg_time_sys = 0.0

    results = {
        'final_clock': sim_time,
        'healed': state['patientsHealed'],
        'arrived': state['totalArrivals'],

        'prob_triage_empty': prob_triage_empty,
        'prob_beds_empty': prob_beds_empty,
        'prob_both_empty': prob_both_empty,

        'avg_nurse_util': avg_nurse_util,
        'avg_occupied_beds': avg_occupied_beds,

        'critical_arrivals': state['criticalCount'],
        'rejected_critical': state['rejectedCritical'],
        'crit_reject_rate': crit_reject_rate,

        'stable_count': state['stableCount'],
        'prop_treated_home': prop_home,

        'avg_time_in_system': avg_time_sys,
    }

    return event_history, results


# ===============================
# RUN & PRINT
# ===============================
if __name__ == "__main__":
    # Example: let's do 20 healed, with 'full' initial condition
    random.seed(SEED)
    ev_hist, stats = run_simulation(target_healed=200, max_events=9999999, initial_condition='full')

    print("First 20 events (or fewer):")
    for row in ev_hist:
        print(row)

    print("\nFinal Stats:")
    for k,v in stats.items():
        print(f"{k}: {v}")

# ===============================
# RUN MULTIPLE SIMULATIONS & REPORT STATS
# ===============================
# if __name__ == "__main__":
#     num_runs = 20
#     all_results = []
#     base_seed = 4040800189

#     for i in range(num_runs):
#         SEED = base_seed + i  # update seed for each run
#         random.seed(SEED)
#         # Run the simulation with target healed patients = 1000 and 'full' initial condition
#         _, stats = run_simulation(target_healed=200, max_events=9999999, initial_condition='full')
#         all_results.append(stats)

#     # Collect metrics of interest across runs
#     metrics = {
#         "Probability triage empty": [r['prob_triage_empty'] for r in all_results],
#         "Probability beds empty": [r['prob_beds_empty'] for r in all_results],
#         "Probability both empty": [r['prob_both_empty'] for r in all_results],
#         "Proportion of critical patients rejected": [r['crit_reject_rate'] for r in all_results],
#         "Average nurse utilization": [r['avg_nurse_util'] for r in all_results],
#         "Average number of occupied beds": [r['avg_occupied_beds'] for r in all_results],
#         "Proportion of patients treated at home": [r['prop_treated_home'] for r in all_results],
#         "Average time a sick person gets better": [r['avg_time_in_system'] for r in all_results],
#     }

#     # Compute mean and 95% confidence intervals (using t-distribution, t ≈ 2.093 for df=19)
#     t_value = 2.093
#     summary = {}
#     for metric, values in metrics.items():
#         n = len(values)
#         mean_val = sum(values) / n
#         variance = sum((x - mean_val) ** 2 for x in values) / (n - 1)
#         std_error = math.sqrt(variance) / math.sqrt(n)
#         ci_lower = mean_val - t_value * std_error
#         ci_upper = mean_val + t_value * std_error
#         summary[metric] = (mean_val, ci_lower, ci_upper)

#     # Print the summary table
#     print("| Performance Metric                          | Mean    | 95% CI (Lower) | 95% CI (Upper) |")
#     print("|--------------------------------------------|---------|---------------|---------------|")
#     for metric, (mean_val, ci_lower, ci_upper) in summary.items():
#         print(f"| {metric:<42} | {mean_val:7.4f} | {ci_lower:13.4f} | {ci_upper:13.4f} |")